In [ ]:
!pip3 install setfit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 13.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 86.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 M

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
#Load dataset
from datasets import load_dataset, Dataset

dataset = load_dataset("witty-works/data_entschieden_unabhangig_ml", use_auth_token=True)
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/witty-works___csv/witty-works--data_entschieden_unabhangig_ml-ecd83fe85c09a4bf/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'sentence', 'purpose', 'word'],
        num_rows: 82
    })
})

In [ ]:
df = dataset["train"].to_pandas()

In [ ]:
df_train = df[df["purpose"]=="train"]
df_train = df_train[["label", "sentence", "word"]]
#create a subset of dataframe
df_train=df_train.loc[:, 'label':'sentence']
df_train = df_train.reset_index(drop=True)
df_train = Dataset.from_pandas(df_train)

In [ ]:
# test data
df_test = df[df["purpose"]=="test"]
df_test = df_test[["label", "sentence", "word"]]
#create a subset of dataframe
df_test = df_test.loc[:, 'label':'sentence']
df_test = df_test.reset_index(drop=True)
df_test = Dataset.from_pandas(df_test)

In [ ]:
df_test

Dataset({
    features: ['label', 'sentence'],
    num_rows: 12
})

In [ ]:
df_train

Dataset({
    features: ['label', 'sentence'],
    num_rows: 55
})

In [ ]:
from setfit import SetFitModel
model_id = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
model = SetFitModel.from_pretrained(model_id)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [ ]:
model_id2 = "sentence-transformers/distiluse-base-multilingual-cased-v2"
model2 = SetFitModel.from_pretrained(model_id2)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [ ]:
model_id3 = "symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli"
model3 = SetFitModel.from_pretrained(model_id3)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [ ]:
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitTrainer

In [ ]:
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitTrainer

trainer = SetFitTrainer(
    model=model2,
    train_dataset=df_train,
    eval_dataset=df_test,
    loss_class=CosineSimilarityLoss,
    batch_size=16,
    num_epochs=1,
    num_iterations=20,
    column_mapping={"sentence": "text", "label": "label"},
)

In [ ]:
trainer.train()

Applying column mapping to training dataset


Generating Training Pairs:   0%|          | 0/20 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 2200
  Num epochs = 1
  Total optimization steps = 138
  Total train batch size = 16


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/138 [00:00<?, ?it/s]

In [ ]:
metrics = trainer.evaluate()

Applying column mapping to evaluation dataset
***** Running evaluation *****


In [ ]:
metrics

{'accuracy': 0.8333333333333334}

In [ ]:
trainer3 = SetFitTrainer(
    model=model3,
    train_dataset=df_train,
    eval_dataset=df_test,
    loss_class=CosineSimilarityLoss,
    batch_size=16,
    num_epochs=1,
    num_iterations=20,
    column_mapping={"sentence": "text", "label": "label"},
)

In [ ]:
trainer3.train()

Applying column mapping to training dataset


Generating Training Pairs:   0%|          | 0/20 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 2200
  Num epochs = 1
  Total optimization steps = 138
  Total train batch size = 16


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/138 [00:00<?, ?it/s]

In [ ]:
metrics3 = trainer3.evaluate()

Applying column mapping to evaluation dataset
***** Running evaluation *****


In [ ]:
metrics3

{'accuracy': 0.9166666666666666}

In [ ]:
# Run inference
preds = model3(["Ich hoffe, dass dort in Ihrem Sinne entschieden wird.", "Eastman setzt sich entschieden und kontinuierlich dafür ein, die Bedürfnisse seiner. Mitarbeiter zu erfüllen und zu übertreffen.", "Der wichtigste Schritt ist, sich von denen zu lösen und sich emotional von deren Einfluss unabhängig zu machen.", "Sie übernahm auf ihre entschiedene Art das Kommando.", "Sie sind unabhängig, analytisch und fleißig"])
preds

tensor([0, 1, 0, 1, 1])